In [ ]:
import json
import os

import pandas as pd
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
DATASET_DIRECTORY = "../raw/article_bias_prediction"
data = []
for filename in os.listdir(DATASET_DIRECTORY):
    filepath = os.path.join(DATASET_DIRECTORY, filename)
    with open(filepath, "r") as file:
        data.append(json.load(file))

df = pd.DataFrame(data)
df

Dropping useless columns. The content column is
> the processed and tokenized content, which is used as input to the different models

and so the original content can be dropped.

In [ ]:
df = df.drop(columns=["topic", "bias", "source", "url", "date", "authors", "content_original", "source_url", "ID"])

Renaming and reordering columns.

In [ ]:
df = df.rename(columns={"content": "body", "bias_text": "leaning"})
df = df[["title", "body", "leaning"]].copy()

Categorizing the leaning column.

In [ ]:
df["leaning"] = df["leaning"].astype("category")

Dropping rows with missing data.

In [ ]:
df = df.dropna(subset=["body", "leaning"])

Printing duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Inspecting body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

After inspection, articles with bodies shorter than 40 words seem to contain no political value. Removing them.

In [ ]:
body_word_count_lower_bound = 40
df = df[df["body_word_count"] >= body_word_count_lower_bound]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["body_length"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

Body length distribution by leaning.

In [ ]:
plt.figure(figsize=(10, 6))

for leaning in df["leaning"].unique():
    df_leaning = df[df["leaning"] == leaning]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_leaning["body_length"].iloc[::100], df_leaning["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=leaning)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by political leaning")
plt.legend(title="leaning")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/article_bias_prediction.parquet")